# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 11** </center>

---

**Fecha**: 09 Mayo 2025

**Nombre del Equipo**: Arriba Linux

**Integrantes del Equipo**: Tirzah Peniche Barba / Ana Cristina Luna Arellano / Juan Pedro Bihouet

**Profesor**: Dr. Pablo Camarillo Ramirez

In [20]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [21]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Arriba-Linux-MLSpark-Logistic-Regression") \
    .master("spark://d9c3cc2bade8:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext
spark.conf.set("spark.sql.shuffle.partitions", "5")

### Preparación de Datos

In [22]:
from ArribaLinux.spark_utils import SparkUtils

iris_schema = SparkUtils.generate_schema([
    ("Id", "integer"),
    ("SepalLengthCm", "double"),
    ("SepalWidthCm", "double"),
    ("PetalLengthCm", "double"),
    ("PetalWidthCm", "double"),
    ("Species", "string")
])

iris_df = spark.read \
               .schema(iris_schema) \
               .option("header", "true") \
               .csv("/home/jovyan/notebooks/data/Iris.csv")

iris_df.printSchema()
iris_df.show(5, truncate=False)

root
 |-- Id: integer (nullable = true)
 |-- SepalLengthCm: double (nullable = true)
 |-- SepalWidthCm: double (nullable = true)
 |-- PetalLengthCm: double (nullable = true)
 |-- PetalWidthCm: double (nullable = true)
 |-- Species: string (nullable = true)



+---+-------------+------------+-------------+------------+-----------+
|Id |SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|Species    |
+---+-------------+------------+-------------+------------+-----------+
|1  |5.1          |3.5         |1.4          |0.2         |Iris-setosa|
|2  |4.9          |3.0         |1.4          |0.2         |Iris-setosa|
|3  |4.7          |3.2         |1.3          |0.2         |Iris-setosa|
|4  |4.6          |3.1         |1.5          |0.2         |Iris-setosa|
|5  |5.0          |3.6         |1.4          |0.2         |Iris-setosa|
+---+-------------+------------+-------------+------------+-----------+
only showing top 5 rows



### Assemble the features into a single vector column

In [24]:
from pyspark.ml.feature import VectorAssembler, StringIndexer

indexer = StringIndexer(inputCol="Species", outputCol="label")
data_indexed = indexer.fit(iris_df).transform(iris_df)

assembler = VectorAssembler(inputCols=["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"], outputCol="features")
data_with_features = assembler.transform(data_indexed).select("label", "features")

### Split the data into training and test sets 80% training data and 20% testing data

In [25]:
train_df, test_df = data_with_features.randomSplit([0.8, 0.2], seed=57)

### Show the whole dataset

In [26]:
print("Original Dataset")
data_with_features.show()

print("train set")
train_df.show()

Original Dataset
+-----+-----------------+
|label|         features|
+-----+-----------------+
|  0.0|[5.1,3.5,1.4,0.2]|
|  0.0|[4.9,3.0,1.4,0.2]|
|  0.0|[4.7,3.2,1.3,0.2]|
|  0.0|[4.6,3.1,1.5,0.2]|
|  0.0|[5.0,3.6,1.4,0.2]|
|  0.0|[5.4,3.9,1.7,0.4]|
|  0.0|[4.6,3.4,1.4,0.3]|
|  0.0|[5.0,3.4,1.5,0.2]|
|  0.0|[4.4,2.9,1.4,0.2]|
|  0.0|[4.9,3.1,1.5,0.1]|
|  0.0|[5.4,3.7,1.5,0.2]|
|  0.0|[4.8,3.4,1.6,0.2]|
|  0.0|[4.8,3.0,1.4,0.1]|
|  0.0|[4.3,3.0,1.1,0.1]|
|  0.0|[5.8,4.0,1.2,0.2]|
|  0.0|[5.7,4.4,1.5,0.4]|
|  0.0|[5.4,3.9,1.3,0.4]|
|  0.0|[5.1,3.5,1.4,0.3]|
|  0.0|[5.7,3.8,1.7,0.3]|
|  0.0|[5.1,3.8,1.5,0.3]|
+-----+-----------------+
only showing top 20 rows

train set
+-----+-----------------+
|label|         features|
+-----+-----------------+
|  0.0|[4.3,3.0,1.1,0.1]|
|  0.0|[4.4,2.9,1.4,0.2]|
|  0.0|[4.4,3.0,1.3,0.2]|
|  0.0|[4.4,3.2,1.3,0.2]|
|  0.0|[4.5,2.3,1.3,0.3]|
|  0.0|[4.6,3.1,1.5,0.2]|
|  0.0|[4.6,3.2,1.4,0.2]|
|  0.0|[4.6,3.4,1.4,0.3]|
|  0.0|[4.6,3.6,1.0,0.2]|
|  0.0|[4.7

### Create a Decision Tree model

In [27]:
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

# TRAINNING

In [28]:
dt_model = dt.fit(train_df)

print("Decision Tree model summary:{0}".format(dt_model.toDebugString))

Decision Tree model summary:DecisionTreeClassificationModel: uid=DecisionTreeClassifier_42457a3652dd, depth=5, numNodes=15, numClasses=3, numFeatures=4
  If (feature 2 <= 2.45)
   Predict: 0.0
  Else (feature 2 > 2.45)
   If (feature 2 <= 4.85)
    If (feature 3 <= 1.65)
     Predict: 1.0
    Else (feature 3 > 1.65)
     If (feature 0 <= 5.95)
      Predict: 1.0
     Else (feature 0 > 5.95)
      Predict: 2.0
   Else (feature 2 > 4.85)
    If (feature 3 <= 1.75)
     If (feature 2 <= 4.95)
      Predict: 1.0
     Else (feature 2 > 4.95)
      If (feature 3 <= 1.55)
       Predict: 2.0
      Else (feature 3 > 1.55)
       Predict: 1.0
    Else (feature 3 > 1.75)
     Predict: 2.0



# PREDICTIONS

In [29]:
# Use the trained model to make predictions on the test data
predictions = dt_model.transform(test_df)

# Show predictions
predictions.select("features", "prediction").show()

+-----------------+----------+
|         features|prediction|
+-----------------+----------+
|[5.0,3.4,1.6,0.4]|       0.0|
|[5.0,3.5,1.3,0.3]|       0.0|
|[5.1,3.7,1.5,0.4]|       0.0|
|[5.2,3.5,1.5,0.2]|       0.0|
|[5.4,3.9,1.7,0.4]|       0.0|
|[5.7,2.6,3.5,1.0]|       1.0|
|[5.8,2.6,4.0,1.2]|       1.0|
|[5.8,2.7,3.9,1.2]|       1.0|
|[6.1,2.8,4.7,1.2]|       1.0|
|[6.1,2.9,4.7,1.4]|       1.0|
|[6.2,2.9,4.3,1.3]|       1.0|
|[6.3,3.3,4.7,1.6]|       1.0|
|[6.7,3.1,4.7,1.5]|       1.0|
|[4.9,2.5,4.5,1.7]|       1.0|
|[6.0,3.0,4.8,1.8]|       2.0|
|[6.1,3.0,4.9,1.8]|       2.0|
|[6.2,3.4,5.4,2.3]|       2.0|
|[6.3,2.5,5.0,1.9]|       2.0|
|[6.3,2.9,5.6,1.8]|       2.0|
|[6.4,3.2,5.3,2.3]|       2.0|
+-----------------+----------+
only showing top 20 rows



# TESTING


In [30]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label",
                            predictionCol="prediction")

accuracy = evaluator.evaluate(predictions, 
                  {evaluator.metricName: "accuracy"})
print(f"Accuracy: {accuracy}")
precision = evaluator.evaluate(predictions,
                  {evaluator.metricName: "weightedPrecision"})
print(f"Precision: {precision}")
recall = evaluator.evaluate(predictions,
                  {evaluator.metricName: "weightedRecall"})
print(f"Recall: {recall}")
f1 = evaluator.evaluate(predictions,
                {evaluator.metricName: "f1"})
print(f"F1 Score: {f1}")    

Accuracy: 0.9545454545454546
Precision: 0.9595959595959596
Recall: 0.9545454545454546
F1 Score: 0.9545454545454545


# Using LinearSVC classifier and OneVsRest multi classifier

In [31]:
from pyspark.ml.classification import LinearSVC, OneVsRest

lsvc = LinearSVC(maxIter=10, regParam=0.01)
ovr = OneVsRest(classifier=lsvc)

In [32]:
ovr_model = ovr.fit(data_with_features)
predictions_svm = ovr_model.transform(test_df)

In [41]:
!pip install numpy




In [42]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator_svm = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")
accuracy = evaluator_svm.evaluate(predictions_svm, {evaluator_svm.metricName: "accuracy"})
precision = evaluator_svm.evaluate(predictions_svm, {evaluator_svm.metricName: "weightedPrecision"})
recall = evaluator_svm.evaluate(predictions_svm, {evaluator_svm.metricName: "weightedRecall"})
f1_svm = evaluator_svm.evaluate(predictions_svm, {evaluator_svm.metricName: "f1"})

print(f"Accuracy (SVM): {accuracy}")
print(f"Precision (SVM): {precision}")
print(f"Recall (SVM): {recall}")
print(f"F1 Score (SVM): {f1_svm}")


Accuracy (SVM): 0.8636363636363636
Precision (SVM): 0.8662337662337662
Recall (SVM): 0.8636363636363636
F1 Score (SVM): 0.8626794258373206


In [43]:
# Show predictions
predictions.select("features", "prediction").show()

+-----------------+----------+
|         features|prediction|
+-----------------+----------+
|[5.0,3.4,1.6,0.4]|       0.0|
|[5.0,3.5,1.3,0.3]|       0.0|
|[5.1,3.7,1.5,0.4]|       0.0|
|[5.2,3.5,1.5,0.2]|       0.0|
|[5.4,3.9,1.7,0.4]|       0.0|
|[5.7,2.6,3.5,1.0]|       1.0|
|[5.8,2.6,4.0,1.2]|       1.0|
|[5.8,2.7,3.9,1.2]|       1.0|
|[6.1,2.8,4.7,1.2]|       1.0|
|[6.1,2.9,4.7,1.4]|       1.0|
|[6.2,2.9,4.3,1.3]|       1.0|
|[6.3,3.3,4.7,1.6]|       1.0|
|[6.7,3.1,4.7,1.5]|       1.0|
|[4.9,2.5,4.5,1.7]|       1.0|
|[6.0,3.0,4.8,1.8]|       2.0|
|[6.1,3.0,4.9,1.8]|       2.0|
|[6.2,3.4,5.4,2.3]|       2.0|
|[6.3,2.5,5.0,1.9]|       2.0|
|[6.3,2.9,5.6,1.8]|       2.0|
|[6.4,3.2,5.3,2.3]|       2.0|
+-----------------+----------+
only showing top 20 rows



In [44]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction")

# Accuracy
accuracy = evaluator.setMetricName("accuracy").evaluate(predictions)
print(f"Accuracy: {accuracy}")

# Precision
precision = evaluator.setMetricName("weightedPrecision").evaluate(predictions)
print(f"Precision: {precision}")

# Recall
recall = evaluator.setMetricName("weightedRecall").evaluate(predictions)
print(f"Recall: {recall}")

# F1 Score
f1 = evaluator.setMetricName("f1").evaluate(predictions)
print(f"F1 Score: {f1}")


Accuracy: 0.9545454545454546
Precision: 0.9595959595959596
Recall: 0.9545454545454546
F1 Score: 0.9545454545454545


In [45]:
sc.stop()

25/05/09 05:45:05 ERROR Inbox: Ignoring error
java.util.concurrent.RejectedExecutionException: Task java.util.concurrent.ScheduledThreadPoolExecutor$ScheduledFutureTask@82a788[Not completed, task = java.util.concurrent.Executors$RunnableAdapter@6dedcae8[Wrapped task = org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend$StandaloneDriverEndpoint$$anon$2@377feaa3]] rejected from java.util.concurrent.ScheduledThreadPoolExecutor@8b9c75b[Terminated, pool size = 0, active threads = 0, queued tasks = 0, completed tasks = 0]
	at java.base/java.util.concurrent.ThreadPoolExecutor$AbortPolicy.rejectedExecution(ThreadPoolExecutor.java:2065)
	at java.base/java.util.concurrent.ThreadPoolExecutor.reject(ThreadPoolExecutor.java:833)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor.delayedExecute(ScheduledThreadPoolExecutor.java:340)
	at java.base/java.util.concurrent.ScheduledThreadPoolExecutor.schedule(ScheduledThreadPoolExecutor.java:562)
	at org.apache.spark.scheduler.cluster